In [259]:
import sklearn as sk
import numpy as np
import pandas as pd
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
import string 
from string import punctuation
from string import digits
import re


[nltk_data] Downloading package stopwords to /Users/yarin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [325]:
fileName='/Users/yarin/Downloads/hillary-clinton-emails/mailinglist.csv'
with open(fileName) as f:
    lines = f.readlines()

In [326]:
df = pd.read_csv(fileName)

In [327]:
df.columns.values

array(['Unnamed: 0', 'emailId', 'sender', 'senderGender',
       'ExtractedSubject', 'ExtractedBodyText', 'receiver',
       'receiverGender', 'mailLength', 'spaces'], dtype=object)

In [328]:
df2 = df[['ExtractedBodyText','senderGender']]

In [329]:
dateWords = ['sunday','monday','tuesday','wednesday','thursday','friday','saturday'
             ,'january','february','march','april','may','june','july','august','september','october','november','december']
stopWords = stopwords.words('english')


for index, row in df2.iterrows():
    row['ExtractedBodyText'] = row['ExtractedBodyText'].lower()
    row['ExtractedBodyText'] = row['ExtractedBodyText'].translate(None, punctuation)
    row['ExtractedBodyText'] = row['ExtractedBodyText'].translate(None, digits)
    row['ExtractedBodyText'] = ' '.join( [w for w in row['ExtractedBodyText'].split() if len(w)>1] )
    row['ExtractedBodyText'] = ' '.join([word for word in row['ExtractedBodyText'].split() if word not in stopWords])
    row['ExtractedBodyText'] = ' '.join([word for word in row['ExtractedBodyText'].split() if word not in dateWords])



/Users/yarin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:11: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  # This is added back by InteractiveShellApp.init_path()


In [330]:
df2

,ExtractedBodyText,senderGender
0,hrodclintonemailcom pm huma abedin fw latest s...,f
1,pis print • hrodclintonernailcom pm °russorvst...,f
2,hrodclintonemailcorn pm huma abedin fw latest ...,f
3,randolph lawrence sent pm mills cheryl subject...,f
4,asked attend svtc today embassy tripoli first ...,m
5,amazing sullivan jacob sullivaniistategov pm s...,m
6,sidney blumenthal magariaf privat reax sent me...,f
7,pis print sullivan jacob mailtosullivanjjstate...,f
8,us department state case doc date state dept p...,f
9,us department state case doc date state dept p...,f


In [331]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None,preprocessor = None,stop_words = None,max_features = 10000)
vectorizer3 = CountVectorizer(analyzer = "word", tokenizer = None,preprocessor = None,ngram_range=(1,3),stop_words = None,max_features = 10000)
vectorizer2 = TfidfVectorizer(analyzer = "word", tokenizer = None,lowercase=True,preprocessor = None,ngram_range=(1,3),stop_words = None,max_features = 10000)

In [332]:
train_data_features = vectorizer.fit_transform(df2['ExtractedBodyText'])


In [333]:
train_data_features = train_data_features.toarray()


In [334]:
vocab = vectorizer.get_feature_names()
dist = np.sum(train_data_features, axis=0)
print(train_data_features.shape)

(2266, 10000)


In [335]:
from sklearn.ensemble import RandomForestClassifier

np.random.seed(123)
x = np.random.rand(len(df2['ExtractedBodyText'])) < 0.7
train_x = train_data_features[x]
test_x = train_data_features[~x]
train_y = df2.loc[x,"senderGender"]
test_y = df2.loc[~x,"senderGender"]

forest = RandomForestClassifier(n_estimators = 100) 
model = forest.fit( train_x, train_y )
model.score(test_x,test_y)

0.83575581395348841

In [336]:
from sklearn import svm 
clf = svm.LinearSVC() 
clf.fit(train_x, train_y) 
clf.score(test_x,test_y)

0.82267441860465118

In [337]:
from sklearn import neighbors 
knn = neighbors.KNeighborsClassifier(3) 
knn.fit(train_x, train_y) 
knn.score(test_x,test_y)

0.74709302325581395

In [286]:
import numpy as np

In [287]:
import theano

In [288]:
import keras

In [338]:
from keras.utils.data_utils import get_file
##path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
#text = open(path).read()
print('corpus length:', len(df2['ExtractedBodyText']))

('corpus length:', 2266)


In [339]:
females=df2[df2.senderGender == 'f']
males=df2[df2.senderGender == 'm']

females = females["ExtractedBodyText"]
males = males["ExtractedBodyText"]



In [340]:
females = females.str.cat(sep=' ')
males = males.str.cat(sep=' ')

In [341]:
females

'hrodclintonemailcom pm huma abedin fw latest syria aiding qaddafi sid hrc memo syria aiding libya docx pis print pis print \xe2\x80\xa2 hrodclintonernailcom pm \xc2\xb0russorvstategov fw meet rightwing extremist behind antifvluslim film sparked deadly riots meat sent pm subject meet right wing extremist behind antimuslim film sparked deadly riots httemaxbiumenthalcommeettherightwingextremistbehindantimusiimtihnthatsparked deadlyriots sent verizon wireless lte droid us department state case doc date state dept produced house select benghazi comm subject agreement sensitive information redactions foia waiver statecb hrodclintonemailcorn pm huma abedin fw latest syria aiding qaddafi sid hrc memo syria aiding libya docx pis print randolph lawrence sent pm mills cheryl subject dry eye nea including mine remarks really moving chriswas amazing man huge loss know libya coming almost ten years never worked calm cool headed funny diplomat made seem really easy even one hardest places work world

In [342]:
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer

females = text_to_word_sequence(females, lower=False, split=" ")
males = text_to_word_sequence(males, lower=False, split=" ")

fToken = Tokenizer(nb_words=1000,char_level=False)
mToken = Tokenizer(nb_words=1000,char_level=False)
fToken.fit_on_texts(females)
mToken.fit_on_texts(males)

In [343]:
fem_mtx = token.texts_to_matrix(females, mode='binary')
mas_mtx = token.texts_to_matrix(males, mode='binary')
fem_mtx.shape
mas_mtx.shape

(18149, 1000)

In [344]:
input_fem = fem_mtx[:-1]
output_fem = fem_mtx[1:]
input_fem.shape, output_fem.shape

((51802, 1000), (51802, 1000))

In [345]:
input_mas = mas_mtx[:-1]
output_mas = mas_mtx[1:]
input_mas.shape, output_mas.shape

((18148, 1000), (18148, 1000))

In [346]:
#from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

In [347]:
modelF = Sequential()
modelM = Sequential()

modelF.add(Embedding(input_dim=input_fem.shape[1],output_dim= 42, input_length=input_fem.shape[1]))
modelM.add(Embedding(input_dim=input_mas.shape[1],output_dim= 42, input_length=input_mas.shape[1]))

In [348]:
modelF.add(Flatten())
modelM.add(Flatten())

modelF.add(Dense(output_fem.shape[1], activation='softmax'))
modelM.add(Dense(output_mas.shape[1], activation='softmax'))

In [349]:
modelF.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])
modelF.fit(input_fem, y=output_fem, batch_size=300, nb_epoch=2, verbose=1, validation_split=0.2)

Train on 41441 samples, validate on 10361 samples
Epoch 1/2
41441/41441 [==============================] - 181s - loss: 3.8553 - acc: 0.0207 - val_loss: 3.7166 - val_acc: 0.0196
Epoch 2/2
41441/41441 [==============================] - 185s - loss: 3.6334 - acc: 0.0438 - val_loss: 3.6119 - val_acc: 0.0431


In [350]:
modelM.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])
modelM.fit(input_mas, y=output_mas, batch_size=300, nb_epoch=2, verbose=1, validation_split=0.2)

Train on 14518 samples, validate on 3630 samples
Epoch 1/2
14518/14518 [==============================] - 62s - loss: 3.2877 - acc: 0.0033 - val_loss: 2.9764 - val_acc: 0.0052
Epoch 2/2
14518/14518 [==============================] - 62s - loss: 3.1311 - acc: 0.0059 - val_loss: 2.9742 - val_acc: 0.0074


In [351]:
def get_next(text,token,model,fullmtx,fullText):
    tmp = text_to_word_sequence(text, lower=False, split=" ")
    tmp = token.sequences_to_matrix(tmp, mode='binary')
    p = model.predict(tmp)
    next_idx = np.random.random_sample()*500
    return fullText[int(next_idx)]

In [303]:

tmp = text_to_word_sequence(wordF, lower=False, split=" ")
tmp = fToken.sequences_to_matrix(tmp, mode='binary')
p = modelF.predict(tmp)

Exception: Error when checking : expected embedding_input_6 to have shape (None, 1000) but got array with shape (1, 2000)

In [352]:
wordF = "dear"
wordM = "dear"
femGenSenList = []
masGenSenList = []

for index1 in range(0,400):
    f = ""
    m = ""
    senlen = np.random.random()*10+10
    for index2 in range(0,int(senlen)):
        wordF = get_next(wordF,fToken,modelF,fem_mtx,females)
        wordM = get_next(wordM,mToken,modelM,mas_mtx,males)
        f = f + " " + wordF
        m = m + " " + wordM 
    femGenSenList.append(f)
    masGenSenList.append(m)

In [353]:
femGenSenList[0:10]

[' waiver like syria mother memo diplo latest sensitive tactic worth date within trust charge comm one playing',
 ' argument risk campaign never muslim libya roves like deadly antimuslim specter got awash reax farewell jacob',
 ' last well hardest surprised todays state randolph yesterday headed agreement agreement',
 ' stevens information pm generally dept something one benghazi stevens oriented brotherhood sid something tribal pm redactions weekend state',
 ' meet state like behind fw en todays neocons blumenthal agreement antifvluslim',
 ' surrogate foia bogeyman aryls neocons magariaf syria headed redactions subject see',
 ' dept particularly libya mailto solution en arms according doc libya gorgeous surrogate adviser deadlyriots redactions foia fw information foia',
 ' obviously redactions benghazi worth last behind charge agreement us raise week mine',
 ' statecb say pressure meet sidney stuart risk state sidney subject places politico hrodclintonemailcom wrote remarks',
 ' catch

In [148]:
masGenSenList[0:10]

[' already of serve thanks has despite 05 of you Speaking',
 ' PM Thanks for in TO and harder STATE with Jeff does',
 ' WAIVER FYI than But were she been had many To',
 ' WAIVER Cheryl WAIVER Chris Philip leadership 2015 Reines R liaison',
 " NATO's AGREEMENT Just your SCB0045269 visits from H try 202 would 8",
 " Heffern R Jacob 3 doing don't deaths Stevens' FIN look INC leadership the SELECT your STATE Eastern Without",
 ' than Benghazi Reines liaison Prom TO D Amazing F 06 would an M Near STATE',
 ' she Tom example sent COMM not Jacob U 34 Department REDACTIONS Sent institution our for embassy DEPT REDACTIONS I',
 ' as afternoon to job she 2015 knee of of to of side',
 ' Secretary Bill I STATE Sullivanii Prom in would I Victoria Jacob that have from svtc R CC TO STATE']

In [354]:
dfF = pd.DataFrame(femGenSenList)
dfM = pd.DataFrame(masGenSenList)

dfF['gender'] = 'f'
dfM['gender'] = 'm'

dfF.columns = ['ExtractedBodyText','senderGender']
dfM.columns = ['ExtractedBodyText','senderGender']

In [355]:
fileRNN = dfF.append(dfM,ignore_index=True)

In [356]:
fileRNN

,ExtractedBodyText,senderGender
0,waiver like syria mother memo diplo latest se...,f
1,argument risk campaign never muslim libya rov...,f
2,last well hardest surprised todays state rand...,f
3,stevens information pm generally dept somethi...,f
4,meet state like behind fw en todays neocons b...,f
5,surrogate foia bogeyman aryls neocons magaria...,f
6,dept particularly libya mailto solution en ar...,f
7,obviously redactions benghazi worth last behi...,f
8,statecb say pressure meet sidney stuart risk ...,f
9,catch riots benghazi pressure obviously presi...,f


In [357]:
df2['flag'] = 'old'
fileRNN['flag'] = 'new'

/Users/yarin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [358]:
df2 = df2.append(fileRNN)

In [359]:
train_data_features = vectorizer.fit_transform(df2['ExtractedBodyText'])
train_data_features = train_data_features.toarray()

In [360]:
vocab = vectorizer.get_feature_names()
dist = np.sum(train_data_features, axis=0)
print(train_data_features.shape)

(3066, 10000)


In [361]:
old=df2[df2.flag == 'old']
new=df2[df2.flag == 'new']

In [362]:
new = new['senderGender']

In [363]:
train_data_features.shape

(3066, 10000)

In [364]:
t = train_data_features[2266:3066]

In [365]:
t.shape

(800, 10000)

In [366]:
model.score(t,new)

0.51624999999999999

In [367]:

clf.score(t,new)

0.62875000000000003

In [368]:
knn.score(t,new) 

0.51124999999999998